In [1]:
from visualizer import calibration
#TODO: import from waveorder
import visual 
from PyQt5 import QtCore
from skimage.io.collection import alphanumeric_key
from dask import delayed
from glob import glob
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np
import napari
from napari import Viewer
import os
import tifffile
import cv2 as cv
import zarr
import cupy as cp 

%gui qt 
%matplotlib inline
plt.style.use('dark_background')


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [2]:
## Open Napari Viewer
viewer = napari.Viewer()

In [3]:
### Enter the filepath for the data directory
main_dir =r'F:\DATA\20220507_M25_ZW495_6'   #5 fast dynamics of big worm
###Enter the filepath for background images
bg_dir = r'F:\Test_ignore\20220509_M25_background_40fps_15gain_2'
### Enter the filepath for Calibration Dataset or offset file
offset_dir = r'F:\DATA\20220503_M25_1um_yg'

#Get Folders and files
main_dirs = sorted(glob(main_dir + '/CAM*/'), key=alphanumeric_key)
bg_folders = sorted(glob(bg_dir + '/CAM*/'), key=alphanumeric_key)
offset_dir = os.path.join(offset_dir + '/'+'processed_files')
offsets_file = os.path.join(offset_dir,"offsets.csv")
with open(offsets_file) as file_name:
    print("loading offsets")
    offsets = np.loadtxt(file_name, delimiter=",")
# Load offsets from desired folder 
processed_files = main_dir + '/'+'processed_files'
if not os.path.exists(processed_files):
    os.makedirs(processed_files)

## Quick Visualizer of the selected planes
# center_planes = calibration.lazy_dask_stack(main_dir,num_cams=5,px_depth='uint16', height=608, width =608)
# napari.add_image(center_planes, name='center_planes',scale=[z_scale,1,1],multiscale=False)

loading offsets


In [4]:
# Scope Parameters
cam_px = 6.0e-6
totalmag = 15.75
zstep = 2e-6

px_size_img = cam_px/totalmag
z_scale = zstep/px_size_img

### Loading Dataset

In [5]:
%%time
# stack = calibration.lazy_dask_stack(main_dir,num_cams=25,px_depth='uint8', height=600, width =960)
stack = calibration.lazy_dask_stack(main_dir,num_cams=25,px_depth='uint16', height=608, width =608)

CPU times: total: 4.02 s
Wall time: 4.02 s


### Get Background Images for subtraction
Get the series of background images to get rid of hot pixels and readout noise

In [20]:
#TODO: get background images of the whole sensor not just the croped region and at 16 bit
bg_stack = calibration.lazy_dask_stack(bg_dir,num_cams=25,px_depth='uint8', height=608, width =608)
bg_mean_stack = da.mean(bg_stack,axis=0)



### Get Calibration from Bead's Files

In [ ]:
# Generate our own shifted datset
with cp.cuda.Device(0):
    # test = stack[:,11:15,:,:]
    test = stack
    t,c,h,w  = test.shape
    np_mip =[]
    for i in range(c):
        cp_stack = cp.array(test[:,i,:,:])
        cp_mip = cp.max(cp_stack, axis=0)
        np_mip.append(cp.asnumpy(cp_mip))
        cp_mip = None
        cp_stack=None
    np_mip = np.array(np_mip)
    np_mip = np.expand_dims(np_mip,axis=0)

# Show the maximum intensity projection
viewer = napari.Viewer()
viewer.add_image(np_mip, name='mip')

NameError: name 'cp' is not defined

In [ ]:
#Go to napari and make a square for alignment
crop_region =viewer.layers['Shapes'].data[0]
template = calibration.create_box_ndarray(crop_region)
template_box = template[0:4,2:4]

# Since OpenCV Match Template takes in uint8 || float32. We can change from uint16 to uint8 easily
np_mip_8bit =visual.im_bit_convert(np_mip,bit=8,norm=True)
min_val, max_val = template_box
t,c = template[0,0:2]
crop_img = np_mip_8bit[t,c, min_val[0]:max_val[0],min_val[1]:max_val[1]]
# viewer.add_image(crop_img,name='crop')


### Perform the actual registration with the chosen method. Look at documentation for other methods..
# TODO: Add capability to select whatever method we want. 
coord =[]
w,h = template.shape  
methods = 'cv.TM_CCOEFF'
# print(w,h)
t,c,h,w = np_mip_8bit.shape

for i in range(c):
    method = eval(methods) # This method parses expression and runs ()
    # Apply template Matching
    # Slide through image and compare template patches.
    # Comparison of best matches is foudn as global minn in SQDIFF or max in CCORR or CCOEF.
    res = cv.matchTemplate(np_mip_8bit[0,i,:,:],crop_img,method)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
        
    bottom_right = (top_left[0] + w, top_left[1] + h)
    #Output would contain the Top left corner and bottom right in case of need to use this rectangle
    coord.append(((top_left[1],top_left[0]),(bottom_right[1],bottom_right[0])))

#For the bead calibration we only use the TL points for alignment
TL_coords = np.array(coord)
TL_coords= TL_coords[0:25,0]

#Coordinates need to then be subtracted to the reference frame (i.e t,c for which rectangle was drawn)
res = []
ref_cam_index = 0
# diff =[]
for i,coordinate in enumerate(TL_coords):
    val = tuple(map(lambda i, j: i - j, TL_coords[ref_cam_index],coordinate))
    res.append(val)
    #If previous values available we can find teh difference between them
    # diff.append(tuple(map(lambda i, j: i + j, val,shift_stack_coord[i])))
# print(diff)
# print(res)
# print(shift_stack_coord)
offset_coordinates = np.array(res)

#Save the Offsets in the folder
np.savetxt(offsets_file, offset_coordinates, delimiter=',')

#Aligning Max Projection to check if coordinates are good
from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset
with cp.cuda.Device(0):
    mip_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(np_mip.shape)
    for i in range(c):
        cp_stack = cp.array(np_mip[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        mip_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    mip_aligned = np.array(mip_aligned)
mip_aligned =  np.moveaxis(mip_aligned,0,1)

#Convert to Dask Array
mip_aligned = da.from_array(mip_aligned)

# Check for alignment using the MIP. If so then proceed to next cell
viewer.add_image(mip_aligned, name='MIP_aligned',scale = [z_scale,1,1])

[[  0.          12.         122.52152243 371.56263532]
 [  0.          12.         122.52152243 430.42760772]
 [  0.          12.         182.63894106 430.42760772]
 [  0.          12.         182.63894106 371.56263532]]
[[123 372]
 [183 430]]
(2, 2)


In [ ]:
#Aligning all the stack after looking at the MIP stack
from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset
with cp.cuda.Device(0):
    stack_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(stack.shape)
    for i in range(c):
        cp_stack = cp.array(stack[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        stack_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    stack_aligned = np.array(stack_aligned)
stack_aligned =  np.moveaxis(stack_aligned,0,1)

#Convert to Dask Array
stack_aligned = da.from_array(stack_aligned)
viewer.add_image(stack_aligned, name='test',scale = [z_scale,1,1])

(25, 3)
(160, 25, 608, 808)


### Align Datasets with Known Offsets


In [22]:
from cupyx.scipy.ndimage import shift
# Generate our own shifted datset

offset_coordinates = offsets
with cp.cuda.Device(0):
    stack_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(stack.shape)
    for i in range(c):
        cp_stack = cp.array(stack[:,i,:,:])
        bg_stack = cp.array(bg_mean_stack[i])
        cp_stack = cp.subtract(cp_stack,bg_stack)
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        stack_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    stack_aligned = np.array(stack_aligned)
stack_aligned =  np.moveaxis(stack_aligned,0,1)

#Convert numpy to dask?
center_cam_compensation = 1.3
da_stack_align = da.from_array(stack_aligned)
da_stack_align[:,12,:,:]= da_stack_align[:,12,:,:]*center_cam_compensation
viewer.add_image(da_stack_align, name='dask_align',scale=[z_scale,1,1], multiscale=False)

(25, 3)
(600, 25, 608, 608)


C:\ProgramData\Anaconda3\envs\m25-py8\lib\site-packages\cupyx\scipy\ndimage\interpolation.py:15: UserWarning: In the current feature the default order of shift is 1. It is different from scipy.ndimage and can change in the future.
  warnings.warn('In the current feature the default order of {} is 1. '


CompileException: C:\Users\yoshi\AppData\Local\Temp\tmp0egtkb2l\d446a609e28fae672544b686e49aa6b6_2.cubin.cu(9): catastrophic error: cannot open source file "math_constants.h"

1 catastrophic error detected in the compilation of "C:\Users\yoshi\AppData\Local\Temp\tmp0egtkb2l\d446a609e28fae672544b686e49aa6b6_2.cubin.cu".
Compilation terminated.


#### SAVE TIFF ALIGNED

In [ ]:
## Save in compressessed lossless format ZARR
import zarr
filename = 'aligned_stack' + '.zarr'
filepath_save = os.path.join(processed_files,filename)
print(filepath_save)
zarr.save(filepath_save,stack_aligned)

In [115]:
## Save in ome tiff files
import tifffile
filename = 'aligned_stack' + '.ome.tif'
filepath_save = os.path.join(processed_files,filename)
print(filepath_save)
tifffile.imsave(filepath_save,stack_aligned)

F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_23/processed_files\aligned_stack.ome.tif


### CROPPING Function for NAPARI
This can be used for getting smaller ROIs. Easiest way to run this is having stack layer and the created shapes layer to crop

In [9]:
#Go to napari and make a square for alignment
crop_region =viewer.layers['Shapes'].data[0]
template = calibration.create_box_ndarray(crop_region)
template_box = template[0:4,2:4]
min_val, max_val = template_box
t,c = template[0,0:2]
crop_img = da_stack_align[:,:, min_val[0]:max_val[0],min_val[1]:max_val[1]]

KeyError: "'Shapes' is not in list"

### Image J Part

In [3]:
### napari viewer must be st before running ImageJ. Viewer was probably called previously so uncomment if needed
# viewer= napari.Viewer()

In [4]:
def start_imagej():
    import imagej
    global ij
    ij = imagej.init( mode='interactive')
    # path = r'C:\Users\yoshi\Documents\Fiji.app'  #Uncomment and replace for FIJI filepath
    # ij = imagej.init(path,mode='interactive')     #Uncomment if using FIJI filepath    
    ij.ui().showUI()
    print(ij.getVersion())
QtCore.QTimer.singleShot(0, start_imagej)

In [15]:
# Get ndarray and send to Java
ij_stack2  = ij.py.to_java(np.array(da_stack_align))
ij.ui().show('PSF', ij_stack2)

In [16]:
# Get the Active Window in imageJ and send back to python
ij.WindowManager.getActiveWindow()
java_img =ij.WindowManager.getCurrentImage()
np_img = ij.py.from_java(java_img)
np_img = np.swapaxes(np_img,1,3)
da_img = da.asarray(np_img)
viewer.add_image(da_img, name='imageJ_img',scale=[z_scale,1,1],multiscale=False)

(60, 342, 413, 25)


#### Functions to save TIFF FIles

In [21]:
import tifffile
filename = 'crop_section' + '.ome.tif'
filepath_save = os.path.join(processed_files,filename)
print(filepath_save)
tifffile.imsave(filepath_save,np_img)

H:\Test\20220503_M25_moreCelegans10pm\crop_section.ome.tif


C:\Users\yoshi\AppData\Local\Temp\ipykernel_17792\34166168.py:6: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(filepath_save,anp2_da)


### Read TIFF FILE

In [5]:
viewer = napari.Viewer()

In [8]:
### For preexisting tiff saved datasets
filename = 'aligned_stack.ome.tif'
filepath_save = os.path.join(processed_files,filename)

In [5]:
filename = 'crop_section' + '.ome.tif'
filepath_save = os.path.join(main_folder,filename)

In [9]:
crop_tiff = tifffile.imread(filepath_save)
print(crop_tiff.shape)
print(type(crop_tiff))
da_crop_tiff = da.from_array(crop_tiff)

viewer.add_image(crop_tiff, name='crop',scale=[z_scale,1,1],contrast_limits=(0,2**16-1))

(735, 25, 608, 608)
<class 'numpy.ndarray'>


<Image layer 'crop' at 0x231a0097d30>

In [9]:
# Align the files
ij_stack2  = ij.py.to_java(np.array(crop_tiff))
ij.ui().show('PSF', ij_stack2)

In [ ]:
ij.WindowManager.getActiveWindow()
a =ij.WindowManager.getCurrentImage()
anp = ij.py.from_java(a)
print(anp.shape)

In [8]:
from napari_animation import Animation
animation = Animation(viewer)
animation.animate('test.gif',canvas_only = False )

### ZARR IMPORT

In [12]:
main_folder = r'H:\Test\20220504_M25_celegansOH_late1'
file_name = main_folder + '/'+'deconvolved_worm' + '.zarr'
zarr_stack = zarr.load(file_name)
da_stack_deconvolved = da.from_array(zarr_stack)


In [14]:
viewer.add_image(da_stack_deconvolved, scale= [z_scale,1,1])

<Image layer 'da_stack_deconvolved [1]' at 0x1351614dbe0>

c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private

### Playing with napari-animation plugin


In [41]:
from napari_animation import AnimationWidget

animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area='right')

ModuleNotFoundError: No module named 'napari_animation'